# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [ ]:
# import numpy and pandas

import pandas as pd
import numpy as np
import scipy.stats as st


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [ ]:
# Your code here:
salaries=pd.read_csv('/content/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [ ]:
# Your code here:
salaries.head()


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [ ]:
# Your code here:

display(salaries.isnull().sum())

salaries.info()  #i found thid new solution

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [ ]:
# Your code here:

salaries['Salary or Hourly'].value_counts()


Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [ ]:
# Your code here:

salaries.groupby('Department').agg({'Name':'count'})


,Name
Department,
ADMIN HEARNG,39
ANIMAL CONTRL,81
AVIATION,1629
BOARD OF ELECTION,107
BOARD OF ETHICS,8
BUDGET & MGMT,46
BUILDINGS,269
BUSINESS AFFAIRS,171
CITY CLERK,84


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [ ]:
# Your code here:
alpha=0.05
hour_salary=salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate']
k=30
stat, p_value=st.ttest_1samp(hour_salary,k)
display(stat)
display(p_value)
p_value<alpha

20.6198057854942

4.3230240486229894e-92

True

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [ ]:
# Your code here:

alpha=0.05

anual_salary= salaries[salaries['Salary or Hourly'] == 'Salary']['Annual Salary']
st.ttest_1samp(anual_salary,86000)


display(stat)
display(p_value)
p_value<alpha

20.6198057854942

4.3230240486229894e-92

True

In [ ]:
anual_salary.mean()

86786.99979015153

Using the `crosstab` function, find the department that has the most hourly workers. 

In [ ]:
# Your code here:

best_department= pd.crosstab(salaries['Department'], salaries['Salary or Hourly'])
best_department['Hourly'].sort_values(ascending=False)

Department
STREETS & SAN            1862
WATER MGMNT              1513
OEMC                     1273
AVIATION                 1082
GENERAL SERVICES          765
TRANSPORTN                725
PUBLIC LIBRARY            299
FAMILY & SUPPORT          287
CITY COUNCIL               64
FINANCE                    44
LAW                        40
ANIMAL CONTRL              19
POLICE                     10
MAYOR'S OFFICE              8
CULTURAL AFFAIRS            7
BUSINESS AFFAIRS            7
HUMAN RESOURCES             4
COMMUNITY DEVELOPMENT       4
HEALTH                      3
BUDGET & MGMT               2
FIRE                        2
PROCUREMENT                 2
DISABILITIES                0
DoIT                        0
COPA                        0
TREASURER                   0
HUMAN RELATIONS             0
CITY CLERK                  0
INSPECTOR GEN               0
BUILDINGS                   0
LICENSE APPL COMM           0
BOARD OF ETHICS             0
BOARD OF ELECTION           0

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [ ]:
salaries[salaries['Department'] == 'STREETS & SAN']['Hourly Rate']

7        35.60
21       21.43
24       35.60
36       36.21
39       35.60
         ...  
33107    35.60
33147    35.60
33149    36.21
33156    22.12
33162      NaN
Name: Hourly Rate, Length: 2198, dtype: float64

In [ ]:
# Your code here:
alpha=0.05
hour_salary_streetsandsan= salaries[salaries['Department'] == 'STREETS & SAN']['Hourly Rate'].dropna()
k=35
stat, p_value=st.ttest_1samp(hour_salary_streetsandsan,k)
display(stat)
display(p_value)
p_value<alpha

-9.567447887848152

3.3378530564707717e-21

True

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [ ]:
salaries.columns

Index(['Name', 'Job Titles', 'Department', 'Full or Part-Time',
       'Salary or Hourly', 'Typical Hours', 'Annual Salary', 'Hourly Rate'],
      dtype='object')

In [ ]:
salaries.head(3)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN


In [ ]:
# Your code here:

hourly = salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate']
mean = np.mean(hourly)
std_error = st.sem(hourly)

st.t.interval(0.95, df=len(salaries) - 1, loc=mean, scale=std_error)

(32.52348867934239, 33.05362675322017)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [ ]:
# Your code here:
salary = salaries[salaries['Salary or Hourly'] == 'Salary']['Annual Salary']
mean = np.mean(salary)
std_error = st.sem(salary)

st.t.interval(0.95, df=len(salaries) - 1, loc=mean, scale=std_error)


(86526.99959172717, 87046.99998857589)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

